### Read .txt file as TSV

In [1]:
import pandas as pd
df = pd.read_csv('../data/wk6/output.txt',sep='\t')
df.head()

,Filename,Label,seg_Start,seg_End,t_ms,HNR05,epoch,soe,Unnamed: 8
0,01_old.mat,ʔm,219.237,392.649,219.0,30.945,0.0,NaN,NaN
1,01_old.mat,ʔm,219.237,392.649,220.0,31.180,0.0,NaN,NaN
2,01_old.mat,ʔm,219.237,392.649,221.0,31.077,0.0,NaN,NaN
3,01_old.mat,ʔm,219.237,392.649,222.0,31.188,0.0,NaN,NaN
4,01_old.mat,ʔm,219.237,392.649,223.0,31.081,1.0,0.0,NaN


### Filter out SOE==0

In [2]:
df_filtered = df[df['soe'].notna()]
print(f"df rows:{len(df)}")
print(f"df_filtered rows:{len(df_filtered)}")
df_filtered.head()

df rows:3273
df_filtered rows:616


,Filename,Label,seg_Start,seg_End,t_ms,HNR05,epoch,soe,Unnamed: 8
4,01_old.mat,ʔm,219.237,392.649,223.0,31.081,1.0,0.000,NaN
30,01_old.mat,ʔm,219.237,392.649,249.0,25.998,1.0,0.000,NaN
55,01_old.mat,ʔm,219.237,392.649,274.0,17.337,1.0,0.001,NaN
71,01_old.mat,ʔm,219.237,392.649,290.0,17.782,1.0,0.001,NaN
83,01_old.mat,ʔm,219.237,392.649,302.0,13.071,1.0,0.001,NaN


### make IPA and group columns

In [3]:
print(df_filtered.columns.tolist())
df_filtered['IPA']=df_filtered['Label']
df_filtered['group_glottalization']=df_filtered['Label'].apply(lambda x: 'glottalization' if 'ʔ' in x else 'no')
df_filtered['group_POA']=df_filtered['IPA'].apply(lambda x: x.replace("ʔ",""))
df_filtered['duration']=df_filtered['seg_End'] - df_filtered['seg_Start']
df_clean = df_filtered[['Filename','duration','HNR05','soe','IPA','group_glottalization','group_POA']]
df_clean.head()

['Filename', 'Label', 'seg_Start', 'seg_End', 't_ms', 'HNR05', 'epoch', 'soe', 'Unnamed: 8']


/tmp/ipykernel_19515/685311562.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['IPA']=df_filtered['Label']
/tmp/ipykernel_19515/685311562.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['group_glottalization']=df_filtered['Label'].apply(lambda x: 'glottalization' if 'ʔ' in x else 'no')
/tmp/ipykernel_19515/685311562.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

,Filename,duration,HNR05,soe,IPA,group_glottalization,group_POA
4,01_old.mat,173.412,31.081,0.000,ʔm,glottalization,m
30,01_old.mat,173.412,25.998,0.000,ʔm,glottalization,m
55,01_old.mat,173.412,17.337,0.001,ʔm,glottalization,m
71,01_old.mat,173.412,17.782,0.001,ʔm,glottalization,m
83,01_old.mat,173.412,13.071,0.001,ʔm,glottalization,m


### Show descriptive statistics: 
- by Place of Articulation
- by Glottalization

In [4]:
# 需要安装 openpyxl: pip install openpyxl
with pd.ExcelWriter('all_statistics.xlsx') as writer:
    # group by glottalization
    numeric_stats_glottal = df_clean.groupby(['group_glottalization']).agg({
        'duration': ['mean', 'std', 'min', 'max'],
        'HNR05': ['mean', 'std', 'min', 'max'],
        'soe': ['mean', 'std', 'min', 'max']
    }).round(1)
    numeric_stats_glottal.to_excel(writer, sheet_name='By_Glottalization')
    
    # group by POA
    numeric_stats_poa = df_clean.groupby(['group_POA']).agg({
        'duration': ['mean', 'std', 'min', 'max'],
        'HNR05': ['mean', 'std', 'min', 'max'],
        'soe': ['mean', 'std', 'min', 'max']
    }).round(1)
    numeric_stats_poa.to_excel(writer, sheet_name='By_POA')
    
    # group by IPA
    numeric_stats_ipa = df_clean.groupby(['IPA']).agg({
        'duration': ['mean', 'std', 'min', 'max'],
        'HNR05': ['mean', 'std', 'min', 'max'],
        'soe': ['mean', 'std', 'min', 'max']
    }).round(1)
    numeric_stats_ipa.to_excel(writer, sheet_name='By_IPA')

print("Saved to: all_statistics.xlsx")

Saved to: all_statistics.xlsx
